In [1]:
import tkinter as tk
from tkinter import scrolledtext
import json
import pickle
import numpy as np
import random
import tensorflow
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()

# Load the preprocessed words and classes
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained model
model = load_model('chatbot_model.h5')

ignore_letters = ["?", "!", ".", ","]

# Load intents from the JSON file
with open("data.json") as file:
    intents_data = json.load(file)
    intents = intents_data["intents"]

In [4]:
class ChatbotGUI(tk.Tk):
    def __init__(self):
        with open("data.json") as file:
            self.intents_data = json.load(file)
            self.intents = self.intents_data["intents"]
        tk.Tk.__init__(self)
        self.title("Chatbot")
        self.geometry("500x500")
        self.configure(bg='#ADD8E6')  # change background color

        self.text_box = scrolledtext.ScrolledText(self, state='disabled', fg='#000080', font=('Helvetica', 12))
        self.text_box.pack(pady=15)

        self.entry = tk.Entry(self, fg='#000080', font=('Helvetica', 12))
        self.entry.pack(pady=15)

        self.button = tk.Button(self, text="Send", command=self.process_user_input, bg='#4B0082', fg='white', font=('Helvetica', 12))
        self.button.pack()

        self.symptoms_input = ""
        self.possible_diseases = []
        self.possible_treatments = {}

        self.greet_user()

    def preprocess_input(self, input_text):
        word_list = nltk.word_tokenize(input_text)
        word_list = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in ignore_letters]
        bag = [1 if word in word_list else 0 for word in words]
        return np.array(bag)

    def extract_symptoms(self, input_text):
        word_list = nltk.word_tokenize(input_text)
        symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in ignore_letters]
        return symptoms

    def predict_intent(self, input_text):
        processed_input = self.preprocess_input(input_text)
        result = model.predict(np.array([processed_input]))[0]
        max_confidence = np.max(result)
        predicted_indices = np.where(result == max_confidence)[0]
        intents_with_max_confidence = [classes[i] for i in predicted_indices]
        return intents_with_max_confidence, max_confidence

    def greet_user(self):
        greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
        self.write_to_text_box("Chatbot: " + random.choice(greetings))

    def write_to_text_box(self, content):
        self.text_box.configure(state='normal')
        if content.startswith("User: "):
            self.text_box.insert('end', content + '\n', 'user')
        else:
            self.text_box.insert('end', content + '\n', 'chatbot')
        self.text_box.configure(state='disabled')
        self.text_box.tag_config('user', foreground='#008000', font=('Helvetica', 12, 'italic'))  # change color and font for user messages
        self.text_box.tag_config('chatbot', foreground='#800080', font=('Helvetica', 12, 'bold'))  # change color and font for chatbot messages

    def resetting_user_input(self, user_input):
        if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set of symptoms" in user_input.lower() or "fresh start" in user_input.lower() or "from beginning" in user_input.lower():
            return True
        else:
            return False

    def chatbot_input(self, user_input):
        intents, confidence = self.predict_intent(user_input)

        # Initialize response and treatments variables
        response = "I'm sorry, I'm unable to determine the problem."
        treatments = []

        # Find the intent details from the intents list
        for intent_data in intents:
            for intent in self.intents_data["intents"]:
                if intent["name"] == intent_data:
                    if "description" in intent:
                        response = intent["description"]
                    if "treatments" in intent:
                        treatments = intent["treatments"]
                    break

        return [("Chatbot:", response), treatments, confidence]

    def get_treatment(self, user_input):
        return "treatment" in user_input.lower() or "cure" in user_input.lower() or \
               "treatments" in user_input.lower() or "cures" in user_input.lower() or \
               "what are the treatments" in user_input.lower()

    def chatbot_respond(self, user_input, treatments):
        if self.get_treatment(user_input):
            treatment_message = "The suggested treatments are: \n- " + ',\n- '.join(treatments)
            return [("Chatbot:", treatment_message)]
        else:
            return [("Chatbot:", "Alright! Feel free to ask if you have more questions.")]

    def is_exit_message(self, user_input):
        exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
        return any(word in user_input.lower() for word in exit_words)

    def possible_disease_enquiry(self, user_input):
        enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
        return any(word in user_input.lower() for word in enquiry_words)

    def get_possible_diseases(self, user_input):
        symptoms = self.extract_symptoms(user_input)
        possible_diseases = []

        for intent in self.intents_data["intents"]:
            for symptom in symptoms:
                if symptom in intent["symptoms"]:
                    possible_diseases.append(intent["name"])
                    break

        return possible_diseases

    def get_possible_diseases_and_treatments(self, user_input):
        symptoms = self.extract_symptoms(user_input)
        possible_diseases = []
        treatments = {}

        for intent in self.intents_data["intents"]:
            for symptom in symptoms:
                if symptom in intent["symptoms"]:
                    possible_diseases.append(intent["name"])
                    treatments[intent["name"]] = intent["treatments"]
                    break

        return possible_diseases, treatments

    def process_user_input(self):
        user_input = self.entry.get()

        # Write user input to chat window
        self.write_to_text_box("User: " + user_input)        
        
        if self.resetting_user_input(user_input):
            self.symptoms_input = ""
            self.possible_diseases = []
            self.possible_treatments = {}
            self.write_to_text_box("Chatbot: Sure, how can I help you")
            self.entry.delete(0, 'end')
            return

        if self.is_exit_message(user_input):
            self.write_to_text_box("Chatbot: You're welcome! Take care!")
            self.entry.delete(0, 'end')
            self.quit()
            return

        self.symptoms_input += " " + user_input
        response, treatments, confidence = self.chatbot_input(self.symptoms_input)

        if confidence < 0.45:
            if self.possible_disease_enquiry(user_input):
                self.possible_diseases, self.possible_treatments = self.get_possible_diseases_and_treatments(self.symptoms_input)
                if len(self.possible_diseases) > 0:
                    self.write_to_text_box(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(self.possible_diseases)}")
                else:
                    self.write_to_text_box("Chatbot: No other predictions as per my knowledge")
                self.entry.delete(0, 'end')
                return

            if self.get_treatment(user_input):
                if len(self.possible_diseases) > 0:
                    for disease in self.possible_diseases:
                        self.write_to_text_box(f"Chatbot: The treatments for {disease} are: {', '.join(self.possible_treatments[disease])}")
                else:
                    self.write_to_text_box("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
                self.entry.delete(0, 'end')
                return

            common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                       "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
            sol = random.choice(common_symptoms_response)
            self.write_to_text_box(f"Chatbot: {sol}")
            self.write_to_text_box(f"Confidence: {confidence}")
            self.entry.delete(0, 'end')
            return

        for item in response:
            self.write_to_text_box(" ".join(item))

        self.write_to_text_box(f"Confidence: {confidence}")

        if self.get_treatment(user_input):
            self.write_to_text_box(f"Chatbot: The treatments are: {treatments}")
            self.entry.delete(0, 'end')
            return

        if self.resetting_user_input(user_input):
            self.symptoms_input = ""
            self.possible_diseases = []
            self.possible_treatments = {}
            self.write_to_text_box("Chatbot: Sure, how can I help you")
            self.entry.delete(0, 'end')
            return  

        if self.is_exit_message(user_input):
            self.write_to_text_box("Chatbot: You're welcome! Take care!")
            self.entry.delete(0, 'end')
            self.quit()
            return



if __name__ == "__main__":
    chatbot = ChatbotGUI()
    chatbot.mainloop()


1/1 [==============================] - 0s 23ms/step
